In [1]:
# General purpose libraries
import boto3
import copy
import csv
import datetime
import json
import numpy as np
import pandas as pd
import s3fs
from collections import defaultdict
import time
import re
import random
from sentence_transformers import SentenceTransformer
import sentencepiece
from scipy.spatial import distance
from json import JSONEncoder
import sys
sys.path.append("/Users/dafirebanks/Projects/policy-data-analyzer/")
sys.path.append("C:/Users/jordi/Documents/GitHub/policy-data-analyzer/")
from tasks.data_loading.src.utils import *

### 1. Set up AWS

In [2]:
def aws_credentials_from_file(f_name):
    with open(f_name, "r") as f:
        creds = json.load(f)
    
    return creds["aws"]["id"], creds["aws"]["secret"]

def aws_credentials(path, filename):
    file = path + filename
    with open(file, 'r') as dict:
        key_dict = json.load(dict)
    for key in key_dict:
        KEY = key
        SECRET = key_dict[key]
    return KEY, SECRET

### 2. Optimized full loop

In [3]:
def aws_credentials(path, filename):
    file = path + filename
    with open(file, 'r') as dict:
        key_dict = json.load(dict)
    for key in key_dict:
        KEY = key
        SECRET = key_dict[key]
    return KEY, SECRET

def aws_credentials_from_file(f_name):
    with open(f_name, "r") as f:
        creds = json.load(f)
    
    return creds["aws"]["id"], creds["aws"]["secret"]

def load_all_sentences(language, s3, bucket_name, init_doc, end_doc):
    policy_dict = {}
    sents_folder = f"{language}_documents/sentences"
    
    for i, obj in enumerate(s3.Bucket(bucket_name).objects.all().filter(Prefix="english_documents/sentences/")):
        
        if not obj.key.endswith("/") and init_doc <= i < end_doc:
            
            serializedObject = obj.get()['Body'].read()
            policy_dict = {**policy_dict, **json.loads(serializedObject)}
            
    return labeled_sentences_from_dataset(policy_dict)

def save_results_as_separate_csv(results_dictionary, queries_dictionary, init_doc, results_limit, aws_id, aws_secret):
    path = "s3://wri-nlp-policy/english_documents/assisted_labeling"
    col_headers = ["sentence_id", "similarity_score", "text"]
    for i, query in enumerate(results_dictionary.keys()):
        filename = f"{path}/query_{queries_dictionary[query]}_{i}_results_{init_doc}.csv"
        pd.DataFrame(results_dictionary[query], columns=col_headers).head(results_limit).to_csv(filename, storage_options={"key": aws_id, "secret": aws_secret})

def labeled_sentences_from_dataset(dataset):
    sentence_tags_dict = {}

    for document in dataset.values():
        sentence_tags_dict.update(document['sentences'])

    return sentence_tags_dict

In [ ]:
# Set up AWS
credentials_file = '/Users/dafirebanks/Documents/credentials.json'
aws_id, aws_secret = aws_credentials_from_file(credentials_file)
region = 'us-east-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = aws_id,
    aws_secret_access_key = aws_secret
)

In [4]:
path = "C:/Users/jordi/Documents/claus/"
filename = "AWS_S3_keys_wri.json"
aws_id, aws_secret = aws_credentials(path, filename)
region = 'us-east-1'

bucket = 'wri-nlp-policy'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = aws_id,
    aws_secret_access_key = aws_secret
)

In [5]:
# Define params
init_at_doc = 4926
end_at_doc = 6568

similarity_threshold = 0
search_results_limit = 500

language = "english"
bucket_name = 'wri-nlp-policy'

transformer_name = 'xlm-r-bert-base-nli-stsb-mean-tokens'
model = SentenceTransformer(transformer_name)


# Get all sentence documents

sentences = load_all_sentences(language, s3, bucket_name, init_at_doc, end_at_doc )

# Define queries
path = "../../input/"
filename = "English_queries.xlsx"
file = path + filename
df = pd.read_excel(file, engine='openpyxl', sheet_name = "Hoja1", usecols = "A:C")

queries = {}
for index, row in df.iterrows():
    queries[row['Query sentence']] = row['Policy instrument']



# Calculate and store query embeddings
query_embeddings = dict(zip(queries, [model.encode(query.lower(), show_progress_bar=False) for query in queries]))

# For each sentence, calculate its embedding, and store the similarity
query_similarities = defaultdict(list)

i = 0
for sentence_id, sentence in sentences.items():
    sentence_embedding = model.encode(sentence['text'].lower(), show_progress_bar=False)
    i += 1
    if i % 100 == 0:
        print(i)
    
    for query_text, query_embedding in query_embeddings.items():
        score = round(1 - distance.cosine(sentence_embedding, query_embedding), 4)
        if score > similarity_threshold:
            query_similarities[query_text].append([sentence_id, score, sentences[sentence_id]['text']])
            
# Sort results by similarity score
for query in query_similarities:
    query_similarities[query] = sorted(query_similarities[query], key = lambda x : x[1], reverse=True)
    
# Store results
save_results_as_separate_csv(query_similarities, queries, init_at_doc, search_results_limit, aws_id, aws_secret)


RuntimeError: CUDA error: out of memory